### TODO

1. lambda functions to create quote and order notification messages
2. implement threads for handling live streaming
  - Design json message structure for each handler
  - Implement decoder for json messages for each handler
3. Implement trade job - init, scanner, order
4. Implement user request handler


#### Future
1. Define function to restart a stopped thread
2. Watchdog implementation to resume processes
3. Implementation of user initiated aborts and restart

## Thread functions

In [2]:
from lib.multitasking_lib import *
#from lib.kite_helper_lib import *
import sys

In [3]:
# Long Strategies
def long_indicators(s):
    temp_df = 0
    #temp_df = temp_df | (REF(s.fastk,1) <=20)&(s.fastk>20)
    #temp_df = temp_df | CROSSOVER(s.fastk, s.fastd)
    #temp_df = temp_df | (REF(s.macd,1) <=0)&(s.macd>0)
    #temp_df = temp_df | CROSSOVER(s.macd, s.macdsignal)
    temp_df = temp_df | (REF(s.rsi,1) <=40)
    #temp_df = temp_df | (s.CLOSE.shift(1) == s.min.shift(1)) & (s.CLOSE > s.OPEN)
    #temp_df = temp_df & CROSSOVER(s.BBB, s.CLOSE)

    return pd.DataFrame(  temp_df, columns=["buy"] )

# Short Strategies
def short_indicators(s):
    temp_df = 0
    #temp_df = temp_df | (REF(s.fastk,1) >=80)&(s.fastk<80)
    #temp_df = temp_df | CROSSOVER(s.fastd, s.fastk)
    #temp_df = temp_df | (REF(s.macd,1) >=0)&(s.macd<0)
    #temp_df = temp_df | CROSSOVER(s.macdsignal, s.macd)
    temp_df = temp_df | (REF(s.rsi,1) >=60)
    #temp_df = temp_df & CROSSOVER(s.CLOSE, s.BBT)
    return pd.DataFrame( temp_df, columns=["sell"])



def long_breakout(s):
    temp_df = s.haCLOSE >= s.BBT.shift(1)
    temp_df = temp_df | (s.haCLOSE >= s.haOPEN.shift(2))
    temp_df = temp_df & ( CROSSOVER(s.OPEN, s.BBT) | CROSSOVER(s.OPEN, s.BBB) | CROSSOVER(s.OPEN, s.BBM))
    return pd.DataFrame( temp_df , columns=["buy"])

def short_breakout(s):
    temp_df = s.haOPEN <= s.BBB.shift(1)
    temp_df = temp_df | (s.haCLOSE <= s.haOPEN.shift(2))
    temp_df = temp_df & ( CROSSOVER(s.BBT, s.OPEN) | CROSSOVER(s.BBB, s.OPEN) | CROSSOVER(s.BBM, s.OPEN))
    return pd.DataFrame( temp_df , columns=["sell"])

def long_ha(s):
    temp_df = (REF(s.haCLOSE,3) < REF(s.haOPEN,3)) & (REF(s.haCLOSE,2) < REF(s.haOPEN,2)) & (REF(s.haCLOSE,1) > REF(s.haOPEN,1)) & (s.haCLOSE > s.haOPEN)  
    #temp_df = temp_df & (s.rsi < 40 )
    return pd.DataFrame( temp_df , columns=["buy"])

def short_ha(s):
    temp_df = (REF(s.haCLOSE,3) > REF(s.haOPEN,3)) & (REF(s.haCLOSE,2) > REF(s.haOPEN,2)) & (REF(s.haCLOSE,1) < REF(s.haOPEN,1)) & (s.haCLOSE < s.haOPEN)
    #temp_df = temp_df & (s.rsi > 60 )
    return pd.DataFrame( temp_df , columns=["sell"])
    


In [10]:
# ====== Tradescript Wrapper =======
# Methods

TREND_UP = lambda a,b: ROC(a, b) >= 0.1
TREND_DOWN = lambda a,b: ROC(a, b) <= -0.1

import traceback
#TREND_UP = lambda a,b: a > MAX(REF(a,1),b)
#TREND_DOWN = lambda a,b: a < MIN(REF(a,1),b)

CROSSOVER = lambda a, b: (REF(a,1)<=REF(b,1)) & (a > b)

#Heikin Asi
def HAIKINASI(ohlc_data_df):
    REF = lambda key, i: ohlc_get(ohlc_data_df.shift(i), key)
    
    OPEN  = ohlc_data_df['open']
    HIGH  = ohlc_data_df['high']
    LOW   = ohlc_data_df['low']
    CLOSE = ohlc_data_df['close']
    
    haOPEN  = (OPEN.shift(1) + CLOSE.shift(1))/2
    haHIGH  = pd.DataFrame([HIGH,OPEN,CLOSE]).max(axis = 0, skipna = True)
    haLOW   = pd.DataFrame([LOW,OPEN,CLOSE]).min(axis = 0, skipna = True)
    haCLOSE = (OPEN+HIGH+LOW+CLOSE)/4
    
    return (haOPEN, haHIGH, haLOW, haCLOSE)

ohlc_get = lambda df, key: df.iloc[-1][key]
REF = lambda df, i: df.iloc[-i-1]
def myalgo(ohlc_data_df): 
    ohlc_data_temp = ohlc_data_df.tail(30)
    
    OPEN = ohlc_data_temp['open']
    CLOSE = ohlc_data_temp['close']
    HIGH = ohlc_data_temp['high']
    LOW = ohlc_data_temp['low']
    VOLUME = ohlc_data_temp['volume']
    
    (haOPEN, haHIGH, haLOW, haCLOSE) = HAIKINASI(ohlc_data_temp)
    
    if (REF(haCLOSE,2) < REF(haOPEN,2)) and (REF(haCLOSE,1) < REF(haOPEN,1)) and (REF(haCLOSE,0) > REF(haOPEN,0)): 
        return "BUY"
    elif (REF(haCLOSE,2) > REF(haOPEN,2)) and (REF(haCLOSE,1) > REF(haOPEN,1)) and (REF(haCLOSE,0) < REF(haOPEN,0)): 
        return "SELL"
    
    return "WAIT" #"BUY"|"SELL"

#ohlc_df = pd.read_json(conn.hget(stock+'_state','ohlc'))
#myalgo(ohlc_df)

In [12]:
# Shut down
conn.publish('kite_simulator','stop')
conn.publish('trade_handler','stop')
freedom.job.terminate()
print(freedom.job.is_alive())
conn.pubsub_channels()

False


['backtest']

## Definition of handler functions

In [13]:

def trade_handler(manager, msg):
    pdebug('trade_handler: {}'.format(msg))
    # Step 1: Blocking call to msgBufferQueue and notificationQueue
    conn.xtrim('msgBufferQueue',maxlen=0, approximate=False)
    conn.xtrim('notificationQueue',maxlen=0, approximate=False)
    while(True):
        msg_q = conn.xread({'msgBufferQueue':'$','notificationQueue':'$'}, block=0, count=100)
        msgs_q = conn.xread({'msgBufferQueue':'0','notificationQueue':'0'}, block=1, count=100)
        conn.xtrim('msgBufferQueue',maxlen=0, approximate=False)
        conn.xtrim('notificationQueue',maxlen=0, approximate=False)
        
        # Step 2: Process notifications: Start a worker thread for each notification
        
        #TODO
        
        # Step 3: Process tick: Start a worker thread for each msg        
        for msg in msgs_q[0][1]:
            pdebug('trade_handler: {}'.format(msg[1]['msg']))
            
            try:
                data = json.loads(msg[1]['msg'])
            except:
                perror("Un-supported message: {} : {}".format(msg, sys.exc_info()[0]))
                break

            for key in data.keys():
                stock = key.split(':')[1]
                exchange = key.split(':')[0]

            hash_key = stock+'_state'
            freq = conn.hget(hash_key,'freq')
            state = conn.hget(hash_key,'state')


            temp_df = msg_to_ohlc(data)
            if state == 'INIT': # State: Init: Load historical data from cache
                # 1: Populate Redis buffer stock+"OHLCBuffer" with historical data
                toDate = (temp_df.index[0] - timedelta(days=1)).strftime('%Y-%m-%d')
                fromDate = (temp_df.index[0] - timedelta(days=no_of_hist_candles)).strftime('%Y-%m-%d')
                ohlc_data = getData(stock, fromDate, toDate, exchange, freq, False, stock)
            else: # Load data from OHLC buffer in hash
                ohlc_data = pd.read_json(conn.hget(hash_key, 'ohlc'))

            ohlc_data = ohlc_data.append(temp_df)
            conn.hset(hash_key,'ohlc',ohlc_data.to_json())

            # Add to OHLCBuffer in hash
            manager.add(stock, trade_job, False, hash_key)
            pdebug(msg[0])
            
# A thread function to process notifications and tick
algo_idle = myalgo
algo_long_so = myalgo
algo_short_so = myalgo

trade_lock = Lock() #TODO: make lock per stockname
def trade_job(hash_key):
    pdebug('trade_job: {}'.format(hash_key))
    
    trade_lock.acquire()
    # Step 1.1: Get stock name from the message    
    
    # Step 1.2: Get state for the stock from the redis
    state = conn.hget(hash_key,'state')
    if not state:
        return
    stock = conn.hget(hash_key,'stock')
    freq = conn.hget(hash_key,'freq')
    pdebug("{}: {}: {}".format(hash_key, stock, state ))
    ohlc_df = pd.read_json(conn.hget(hash_key,'ohlc'))
    
    last_processed = ohlc_df.index[-1].strftime('%Y-%m-%d')
    pdebug("{}=>{}".format(last_processed,conn.hget(hash_key,'last_processed')))
    
    if last_processed == conn.hget(hash_key,'last_processed'):   
        trade_lock.release()
        return
    else:
        conn.hset(hash_key,'last_processed',last_processed)
    
    #print('{}:{}'.format(stock,ohlc_df.index[-1]))
    
    # Step 2: Switch to appropriate state machine based on current state
    if state == 'INIT': # State: Init
        # 1: Populate Redis buffer stock+"OHLCBuffer" with historical data
            # Done inside thread handler
        
        # 2: Set state to Scanning
        conn.hset(hash_key,'state','SCANNING')
        pass
    
    elif state == 'SCANNING':  # State: Scanning
        # 1: Run trading algorithm for entering trade
        tradeDecision = algo_idle(ohlc_df)
        
        # 2: If Algo returns Buy: set State to 'Pending Order: Long'
        if tradeDecision=="BUY":
            logtrade("BUY : {} : {} -> {}".format(last_processed, stock, ohlc_get(ohlc_df,'close')))
            conn.hset(hash_key,'state','PO:LONG')
        
        # 3: If Algo returns Sell: set State to 'Pending Order: Short'
        elif tradeDecision=="SELL":
            logtrade("SELL: {} : {} -> {}".format(last_processed, stock, ohlc_get(ohlc_df,'close')))
            conn.hset(hash_key,'state','PO:SHORT')
        
        # 4: Update TradeMetaData: Push order details to OrderQueue
    
    elif state == 'PO:LONG': # State: Pending Order: Long
    
        # 1: On Fill: set State to Long
        conn.hset(hash_key,'state','LONG')
        pass
    
    
    elif state == 'PO:SHORT': # State: Pending Order: Short
    
        # 1: On Fill: set State to Short
        conn.hset(hash_key,'state','SHORT')
        pass
    
    
    elif state == 'LONG': # State: Long
    
        # 1: If notification for AutoSquare Off: set state to init
        
        # 2: Else run trading algorithm for square off
        
        tradeDecision = algo_long_so(ohlc_df)
        if tradeDecision == "SELL":
            logtrade("SO-S: {} : {} -> {}".format(last_processed, stock, ohlc_get(ohlc_df,'close')))

            # 3: If algo returns square off: then push square off details to OrderQueue, set state to 'Awaiting Square Off'   
            conn.hset(hash_key,'state','SQUAREOFF')
    
    
    elif state == 'SHORT': # State: Short
    
        # 1: If notification for AutoSquare Off: set state to init
        
        # 2: Else run trading algorithm for square off
        tradeDecision = algo_short_so(ohlc_df)
        
        if tradeDecision == "BUY":
            logtrade("SO-B: {} : {} -> {}".format(last_processed, stock, ohlc_get(ohlc_df,'close')))
        
            # 3: If algo returns square off: then push square off details to OrderQueue, set state to 'Awaiting Square Off'
    
            conn.hset(hash_key,'state','SQUAREOFF')
        
    elif state == 'SQUAREOFF':  # State: Awaiting Square Off
        
        conn.hset(hash_key,'state','INIT')
        pass
   
        # 1: On Fill notification: set state to Init

    trade_lock.release()



In [14]:
#freedom_init = threadManager("freedom_init", ["freedom_init"], [freedom_init])   

#freedom = threadManager("freedom", ["user_requests_handler", "kite_simulator", "backtest_handler", "trade_handler","order_handler"], 
#                        [user_requests_handler, kite_simulator, backtest_handler, trade_handler, order_handler])

logger.setLevel(21)
freedom = threadManager("freedom", [ "kite_simulator", "trade_handler", "order_handler"], 
                        [kite_simulator, trade_handler, order_handler])


TRADE: SELL: 2019-01-10 : INFY -> 679.7
TRADE: SO-B: 2019-01-25 : INFY -> 730.35
TRADE: SELL: 2019-02-05 : INFY -> 754.85
TRADE: SO-B: 2019-02-20 : INFY -> 740.7
TRADE: SELL: 2019-02-27 : INFY -> 735.25
TRADE: SO-B: 2019-03-01 : INFY -> 741.9
TRADE: BUY : 2019-03-13 : INFY -> 708.45
TRADE: SO-S: 2019-03-18 : INFY -> 710.2
TRADE: SELL: 2019-03-25 : INFY -> 735.25
TRADE: SO-B: 2019-03-28 : INFY -> 737.8
TRADE: SELL: 2019-04-03 : INFY -> 753.3


In [15]:

# Unit Test#1
freq='day'
exchange='NSE'
fromDate = '2019-01-01'
toDate = '2019-05-03'
stock='INFY'

state = 'INIT'
hash_key = stock+'_state'

try:
    all_keys = list(conn.hgetall(hash_key).keys())
    conn.hdel(hash_key,*all_keys)
except:
    pass

#conn.hmset(hash_key, {'state':state,'stock':stock, 'qty':0,'price':0,'algo':'','freq':'day','so':0,'target':0})

msg = json.dumps({'stock': stock, 'fromDate':fromDate,'toDate':toDate, 'exchange':exchange, 'freq':freq})
conn.publish('trade_handler','start')
msgid2 = conn.publish('kite_simulator',msg)


In [28]:
# Unit Test#2
stock = 'NDTV'
msg = json.dumps({'stock': stock, 'fromDate':fromDate,'toDate':toDate, 'exchange':exchange, 'freq':freq})
conn.publish('trade_handler','start')
msgid2 = conn.publish('kite_simulator',msg)
notification_despatcher(None, 'msg')

In [9]:
notification_despatcher(None, 'stop')

In [41]:
# Shut down
conn.publish('kite_simulator','stop')
conn.publish('trade_handler','stop')
freedom.job.terminate()
print(freedom.job.is_alive())
conn.pubsub_channels()

True


['trade_handler', 'backtest', 'order_handler']

In [42]:
conn.pubsub_channels()

['backtest']

In [35]:
# Debug
stock='TCS'
#notification_despatcher(None, 'msg')
print(stock)
pd.read_json(conn.hget(stock+'_state','ohlc'))

TCS


,close,high,low,open,volume
2019-01-14,1813.25,1851.00,1810.00,1850.00,3615370
2019-01-15,1867.80,1869.35,1808.00,1810.00,7711653
2019-01-16,1870.15,1875.00,1847.00,1870.00,2359280
2019-01-17,1894.30,1898.00,1871.30,1878.00,2625816
2019-01-18,1900.65,1904.20,1876.00,1881.00,1683728
...,...,...,...,...,...
2019-04-25,2195.00,2205.00,2168.05,2175.00,4211526
2019-04-26,2238.55,2243.95,2193.15,2203.00,2716958
2019-04-30,2260.35,2266.95,2230.10,2238.55,3488395
2019-05-02,2215.40,2259.40,2211.10,2255.00,2457699


In [10]:
pd.set_option('display.max_rows', 50)
pd.read_json(conn.hget(stock+'_state','ohlc')).tail(50)

,close,high,low,open,volume
2019-02-15,282.78,283.72,278.31,282.14,3522673
2019-02-18,282.41,284.81,278.76,284.81,3480075
2019-02-19,273.27,282.26,272.33,280.11,3064358
2019-02-20,279.85,281.20,271.58,273.08,4410484
2019-02-21,281.62,283.31,278.80,280.30,2573085
2019-02-22,284.77,285.71,280.83,281.95,3194664
2019-02-25,289.77,290.75,285.15,286.32,4678470
2019-02-26,291.47,292.44,286.65,289.10,3802537
2019-02-27,281.50,288.61,280.00,285.90,8009017
2019-02-28,277.37,283.23,275.64,281.88,9428265


## Scratchpad

In [1]:
import pandas as pd
import plotly
import plotly.graph_objs as go
from datetime import datetime as dt
from datetime import timedelta
from plotly.subplots import make_subplots
import plotly.graph_objects as go

from lib.multitasking_lib import *
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

In [2]:
logger.setLevel(logging.DEBUG)
stock = 'WIPRO'
fromDate = '2019-1-1'
toDate  = '2019-4-30'
freq = 'day'
exchange = 'NSE'
ohlc_data = getData(stock, fromDate, toDate, exchange, freq, False, stock)

INFO:	Using cache: Not downloading data


In [24]:
plot_layout = {
    'height': 950,
    'plot_bgcolor': 'rgba(0,0,0,0)',
    'template': '...',
    'title': {'text': 'Charts for WIPRO'},
    'xaxis': {'rangeselector': {'buttons': [{'count': 1, 'label': '1h', 'step': 'hour', 'stepmode': 'backward'},
                                            {'count': 3, 'label': '3h', 'step': 'hour', 'stepmode': 'backward'},
                                            {'count': 6, 'label': '1d', 'step': 'hour', 'stepmode': 'backward'},
                                            {'step': 'all'}]},
              'rangeslider': {'visible': False},
              'side': 'bottom'},
    'xaxis2': {'anchor': 'y2', 'domain': [0.0, 1.0], 'matches': 'x5', 'showticklabels': False},
    'xaxis3': {'anchor': 'y3', 'domain': [0.0, 1.0], 'matches': 'x5', 'showticklabels': False},
    'xaxis4': {'anchor': 'y4', 'domain': [0.0, 1.0], 'matches': 'x5', 'showticklabels': False},
    'xaxis5': {'anchor': 'y5', 'domain': [0.0, 1.0]},
    'yaxis': {'anchor': 'x', 'domain': [0.5636363636363636, 1.0], 'side': 'right'},
    'yaxis2': {'anchor': 'x2', 'domain': [0.4663636363636363, 0.5536363636363636]},
    'yaxis3': {'anchor': 'x3', 'domain': [0.19454545454545452, 0.4563636363636363]},
    'yaxis4': {'anchor': 'x4', 'domain': [0.09727272727272726, 0.1845454545454545]},
    'yaxis5': {'anchor': 'x5', 'domain': [0.0, 0.08727272727272727]}
}

#plot_layout

In [33]:
fig.layout = plot_layout

AttributeError: 'dict' object has no attribute 'layout'

In [203]:
fig = make_subplots(rows=3, cols=1, shared_xaxes=True, row_width=[3,1,5], vertical_spacing = 0.01)
fig['layout']={'xaxis':{'rangeselector': {'buttons': [{'count': 1, 'label': '1h', 'step': 'hour', 'stepmode': 'backward'},
                                         {'count': 3, 'label': '3h', 'step': 'hour', 'stepmode': 'backward'},
                                        {'count': 6, 'label': '1d', 'step': 'hour', 'stepmode': 'backward'},
                                            {'step': 'all'}]},
              'rangeslider': {'visible': False}, 'side': 'bottom'}, 
              'xaxis2': {'anchor': 'y2', 'domain': [0.0, 1.0], 'matches': 'x', 'showticklabels': False},
              'xaxis3': {'anchor': 'y3', 'domain': [0.0, 1.0], 'matches': 'x', 'showticklabels': True},
              'yaxis' : {'anchor': 'x', 'domain': [0.45, 1.0], 'side': 'right', 'linecolor':'black', 'ticks':'inside'},
              'yaxis2': {'anchor': 'x2', 'domain': [0.2, 0.43], 'side': 'right', 'linecolor':'black', 'ticks':'inside'},
              'yaxis3': {'anchor': 'x3', 'domain': [0.0, 0.19], 'side': 'right', 'range':[0,100], 'tickvals':[0,30,70,100], 'ticks':'inside','gridcolor':'black', 'showgrid':True, 'linecolor':'black'},
              'height': 650, 'plot_bgcolor': 'rgba(0,0,0,0)','title': {'text': 'Charts for WIPRO'}}



scatter = lambda df, key, title, c, fill='none', fillcolor="rgba(0,40,100,0.02)": go.Scatter(x=df.index.astype('str'), y=df[key], name=title, line=dict(color=c),showlegend=False, fill = fill, fillcolor=fillcolor)
bar =  lambda df, key, title, c: go.Bar(x=df.index.astype('str'), y=df[key], name=title, marker=dict(color=c),showlegend=False)

def plot_2_lines_1_bar(fig, df, d1, t1='', c1='black', d2=None, t2='', c2='red', d3=None, t3='', c3='grey', pos = 1):
    trace1 = scatter(df, d1, t1, c1)
    fig.append_trace(trace1, pos, 1)
    
    if d2:
        trace2 = scatter(df, d2, t2, c2)
        fig.append_trace(trace2, pos, 1)
    if d3:
        trace3 = bar(df, d3, t3, c3)
        fig.append_trace(trace3, pos, 1)
    
    return fig

def plot_3_lines(fig, df, d1, t1='', c1='black', d2=None, t2='', c2='red', d3=None, t3='', c3='grey', pos = 1, fill=False, fillcolor="rgba(0,40,100,0.02)"):
    trace1 = scatter(df, d1, t1, c1)
    fig.append_trace(trace1, pos, 1)
    
    filltonexty = "none"
    if fill==True:
        filltonexty = 'tonexty'
    
    if d2:
        trace2 = scatter(df, d2, t2, c2, fill=filltonexty, fillcolor=fillcolor)
        fig.append_trace(trace2, pos, 1)
        
    if d3:
        trace3 = scatter(df, d3, t3, c3, fill=filltonexty, fillcolor=fillcolor)
        fig.append_trace(trace3, pos, 1)
    
    return fig


plot_candle = lambda fig, df, pos = 1: fig.append_trace( go.Candlestick(x=df.index.astype('str'), open=df.open, high=df.high, low=df.low, close=df.close, name="Candlestick", showlegend=False), pos, 1)
plot_macd = lambda fig, df, pos = 1: plot_2_lines_1_bar(fig, df, 'macd', 'MACD' ,'black', 'macdsignal','MACD Signal', 'red', 'macdhist','MACD Histogram', 'grey', pos)
plot_rsi =  lambda fig, df, pos = 1: plot_2_lines_1_bar(fig, df, 'rsi', 'RSI' ,'black', pos=pos)
plot_bbb =  lambda fig, df, pos = 1, fill=True, fillcolor="rgba(0,40,100,0.02)": plot_3_lines(fig, df, 'bbt', 'Top' ,'lightgrey', 'bbb','Bottom', 'lightgrey', 'bbm','Middle', 'lightgrey', pos, fill, fillcolor)



price = ohlc_data
price['macd'], price['macdsignal'], price['macdhist'] = MACDEXT(price.close, fastperiod=12, slowperiod=26, signalperiod=9, fastmatype=1, slowmatype=1,signalmatype=1)
price['rsi'] = RSI(price.close, timeperiod=14)
price['bbt'], price['bbm'], price['bbb'] = BBANDS(price.close, timeperiod=20, nbdevup=1.6, nbdevdn=1.6, matype=0)

price['buy'] = ''
price['sell'] = ''
price['buy'] = ohlc_data['close'][30:40]
price['sell'] = ohlc_data['close'][50:51]

def plot_trade(fig, df, pos=1):
    fig.append_trace(go.Scatter(x=df.index.astype('str'), y=df['buy']*0.8 ,  mode='markers', marker=dict(color='green'),showlegend=False, hovertext=df['buy']), pos, 1)
    fig.append_trace(go.Scatter(x=df.index.astype('str'), y=df['sell']*1.2,  mode='markers', marker=dict(color='red'),showlegend=False, hovertext=df['sell']), pos, 1)
    return fig
    
plot_candle(fig, price, 1)

fig = plot_trade(fig, price, 1)
fig = plot_bbb(fig, price, 1)
fig = plot_macd(fig, price, 2)
fig = plot_rsi(fig, price, 3)


init_notebook_mode(connected=True)
iplot(fig)

In [51]:
fig = make_subplots(rows=3, cols=1, shared_xaxes=True, row_width=[3,1,5], vertical_spacing = 0.01)
fig['layout']

Layout({
    'template': '...',
    'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'matches': 'x3', 'showticklabels': False},
    'xaxis2': {'anchor': 'y2', 'domain': [0.0, 1.0], 'matches': 'x3', 'showticklabels': False},
    'xaxis3': {'anchor': 'y3', 'domain': [0.0, 1.0]},
    'yaxis': {'anchor': 'x', 'domain': [0.45555555555555555, 1.0]},
    'yaxis2': {'anchor': 'x2', 'domain': [0.33666666666666667, 0.44555555555555554]},
    'yaxis3': {'anchor': 'x3', 'domain': [0.0, 0.32666666666666666]}
})

In [ ]:

 
if False:
    msg_ohlc = {"NSE:WIPRO": {"ohlc": {"date": "2019-1-1", "open": 1896.0, "high": 1910.0, "low": 1885.0, "close": 1902.8, "volume": 1094883.0}}}

    stock='WIPRO'
    state = 'INIT'
    hash_key = stock+'_state'

    try:
        all_keys = list(conn.hgetall(hash_key).keys())
        conn.hdel(hash_key,*all_keys)
    except:
        pass

    conn.hmset(hash_key, {'state':state,'stock':stock, 'qty':0,'price':0,'algo':'','freq':'day','so':0,'target':0})

    logger.setLevel(logging.INFO)
    trade_job(json.dumps(msg_ohlc))
    #trade_job(json.dumps(msg_ohlc))
    #trade_job(json.dumps(msg_ohlc))
    #trade_job(json.dumps(msg_ohlc))
    #trade_job(json.dumps(msg_ohlc))
    #trade_job(json.dumps(msg_ohlc))

if False:
    msg_ohlc = {"NSE:WIPRO": {"ohlc": {"date": "2019-1-3", "open": 1896.0, "high": 1910.0, "low": 1885.0, "close": 1902.8, "volume": 1094883.0}}}
    trade_job(json.dumps(msg_ohlc))